In [7]:
!wget -O ravdess.zip "https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip?download=1"


--2025-04-03 14:08:52--  https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1188976/files/Audio_Speech_Actors_01-24.zip [following]
--2025-04-03 14:08:52--  https://zenodo.org/records/1188976/files/Audio_Speech_Actors_01-24.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 208468073 (199M) [application/octet-stream]
Saving to: ‘ravdess.zip’

ravdess.zip         100%[===================>] 198.81M  30.5MB/s    in 7.3s    

2025-04-03 14:09:00 (27.1 MB/s) - ‘ravdess.zip’ saved [208468073/208468073]



In [8]:
!unzip -q ravdess.zip -d /content/ravdess/


In [6]:
!ls /content/ravdess/


ls: cannot access '/content/ravdess/': No such file or directory


In [5]:
!find /content/ravdess/ -type f -name "*.wav" | head -10


/content/ravdess/Actor_03/03-01-02-01-02-01-03.wav
/content/ravdess/Actor_03/03-01-06-02-01-01-03.wav
/content/ravdess/Actor_03/03-01-06-01-02-01-03.wav
/content/ravdess/Actor_03/03-01-07-01-01-02-03.wav
/content/ravdess/Actor_03/03-01-02-01-01-02-03.wav
/content/ravdess/Actor_03/03-01-01-01-01-01-03.wav
/content/ravdess/Actor_03/03-01-02-01-02-02-03.wav
/content/ravdess/Actor_03/03-01-04-01-02-01-03.wav
/content/ravdess/Actor_03/03-01-04-01-02-02-03.wav
/content/ravdess/Actor_03/03-01-08-01-01-02-03.wav


In [7]:


import IPython.display as ipd

# Replace with an actual file path
audio_path = "/content/ravdess/Actor_03/03-01-02-01-02-01-03.wav"
ipd.Audio(audio_path)


In [8]:
!find /content/ravdess/ -type f -name "*.wav" | head -10


/content/ravdess/Actor_03/03-01-02-01-02-01-03.wav
/content/ravdess/Actor_03/03-01-06-02-01-01-03.wav
/content/ravdess/Actor_03/03-01-06-01-02-01-03.wav
/content/ravdess/Actor_03/03-01-07-01-01-02-03.wav
/content/ravdess/Actor_03/03-01-02-01-01-02-03.wav
/content/ravdess/Actor_03/03-01-01-01-01-01-03.wav
/content/ravdess/Actor_03/03-01-02-01-02-02-03.wav
/content/ravdess/Actor_03/03-01-04-01-02-01-03.wav
/content/ravdess/Actor_03/03-01-04-01-02-02-03.wav
/content/ravdess/Actor_03/03-01-08-01-01-02-03.wav


In [9]:
import IPython.display as ipd

# Use one of the actual file paths from your list
audio_path = "/content/ravdess/Actor_03/03-01-02-01-02-01-03.wav"

# Play the audio
ipd.Audio(audio_path)


In [11]:
import pandas as pd
import librosa
import os

# Define dataset directory
data_dir = "/content/ravdess/Actor_03"

# Collect all audio file paths
audio_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.wav')]

# Feature extraction function
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)  # Load audio
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Extract MFCC features
    return mfccs.mean(axis=1)  # Take the mean of MFCCs

# Create dataframe
data = []
for file in audio_files:
    label = file.split("/")[-1].split("-")[2]  # Extract emotion label from filename
    features = extract_features(file)
    data.append([features, label])

# Convert to DataFrame
df = pd.DataFrame(data, columns=["features", "label"])

# Print dataset sample
print(df.head())


                                            features label
0  [-670.2019, 95.914825, -2.0398574, 17.43807, -...    02
1  [-416.87708, 93.55297, -13.417342, 14.867507, ...    06
2  [-557.5914, 79.30087, -10.559692, 17.806805, -...    06
3  [-618.37427, 83.005135, -1.6759187, 12.043492,...    07
4  [-665.8279, 84.23869, -1.4056453, 15.918263, 0...    02


In [12]:
# Install necessary libraries
!pip install librosa scikit-learn numpy pandas matplotlib

# Import Libraries
import os
import numpy as np
import librosa
import librosa.display
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Function to extract MFCC features
def extract_mfcc(file_path, n_mfcc=40):
    y, sr = librosa.load(file_path, sr=None)  # Load audio
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)  # Compute MFCCs
    return np.mean(mfcc.T, axis=0)  # Compute mean

# Function to load dataset and extract features
def load_dataset(directory):
    features, labels = [], []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".wav"):
                file_path = os.path.join(root, file)
                try:
                    mfcc_features = extract_mfcc(file_path)
                    emotion_label = int(file.split("-")[2]) - 1  # Extract emotion label
                    features.append(mfcc_features)
                    labels.append(emotion_label)
                except:
                    print(f"Error processing {file}")
    return pd.DataFrame(features), np.array(labels)

# Load dataset
dataset_path = "/content/ravdess"  # Change as per dataset path
X, y = load_dataset(dataset_path)

# Split data into train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM classifier
model = SVC(kernel="rbf", C=1.0, gamma="scale")
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"🎯 Model Accuracy: {accuracy:.2f}")
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))

# Function to predict emotion from new audio
def predict_emotion(file_path):
    mfcc_features = extract_mfcc(file_path).reshape(1, -1)
    predicted_label = model.predict(mfcc_features)[0]
    emotions = {0: "Neutral", 1: "Calm", 2: "Happy", 3: "Sad", 4: "Angry", 5: "Fearful", 6: "Disgust", 7: "Surprised"}
    return emotions[predicted_label]

# Test with a new audio file
file_path = "/content/ravdess/Actor_03/03-01-06-01-02-01-03.wav"  # Change file path
predicted_emotion = predict_emotion(file_path)
print(f"🎭 Predicted Emotion: {predicted_emotion}")


🎯 Model Accuracy: 0.29

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        18
           1       0.25      0.90      0.40        30
           2       0.11      0.12      0.12        40
           3       0.16      0.10      0.12        31
           4       0.41      0.68      0.51        38
           5       0.00      0.00      0.00        45
           6       0.30      0.16      0.21        43
           7       0.48      0.37      0.42        43

    accuracy                           0.29       288
   macro avg       0.22      0.29      0.22       288
weighted avg       0.23      0.29      0.23       288

🎭 Predicted Emotion: Happy


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
import os
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.utils import to_categorical

# 📌 Define RAVDESS Emotion Labels
emotion_labels = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
}

# 📌 Function to Extract MFCC Features
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    return np.mean(mfcc, axis=1)

# 📌 Load Dataset
data_path = "/content/ravdess"
features, labels = [], []

for actor in os.listdir(data_path):
    actor_path = os.path.join(data_path, actor)
    for file in os.listdir(actor_path):
        file_path = os.path.join(actor_path, file)
        emotion_code = file.split("-")[2]
        emotion = emotion_labels.get(emotion_code)

        if emotion:
            mfcc_features = extract_features(file_path)
            features.append(mfcc_features)
            labels.append(emotion)

# Convert to NumPy arrays
X = np.array(features)
y = np.array(labels)

# 📌 Encode Labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)  # One-hot encode labels

# 📌 Standardize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape for CNN/LSTM
X_cnn = X_scaled[..., np.newaxis]  # Add channel dimension for CNN
X_lstm = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)  # Reshape for LSTM

# 📌 Split Dataset (80% Train, 20% Test)
X_train_cnn, X_test_cnn, y_train, y_test = train_test_split(X_cnn, y_categorical, test_size=0.2, random_state=42)
X_train_lstm, X_test_lstm, _, _ = train_test_split(X_lstm, y_categorical, test_size=0.2, random_state=42)

# 📌 Build CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(40, 1)),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='softmax')  # 8 emotion classes
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 📌 Train CNN Model
cnn_model.fit(X_train_cnn, y_train, validation_data=(X_test_cnn, y_test), epochs=50, batch_size=16)

# 📌 Build LSTM Model
lstm_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(40, 1)),
    LSTM(128),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(8, activation='softmax')  # 8 emotion classes
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 📌 Train LSTM Model
lstm_model.fit(X_train_lstm, y_train, validation_data=(X_test_lstm, y_test), epochs=50, batch_size=16)

# 📌 Evaluate Models
cnn_acc = cnn_model.evaluate(X_test_cnn, y_test)[1]
lstm_acc = lstm_model.evaluate(X_test_lstm, y_test)[1]

print(f"🔹 CNN Accuracy: {cnn_acc:.2f}")
print(f"🔹 LSTM Accuracy: {lstm_acc:.2f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.2105 - loss: 2.1170 - val_accuracy: 0.2847 - val_loss: 1.9901
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4152 - loss: 1.6575 - val_accuracy: 0.3576 - val_loss: 1.8944
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4722 - loss: 1.4406 - val_accuracy: 0.3993 - val_loss: 1.7615
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.5367 - loss: 1.3201 - val_accuracy: 0.4618 - val_loss: 1.6038
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6171 - loss: 1.0699 - val_accuracy: 0.5278 - val_loss: 1.4731
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6357 - loss: 0.9798 - val_accuracy: 0.5451 - val_loss: 1.3042
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.6680 - loss: 0.8977 - val_accuracy: 0.5799 - val_loss: 1.1858
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7156 - loss: 0.7525 - val_accuracy: 0.5764 - va

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 9s 58ms/step - accuracy: 0.1511 - loss: 2.0691 - val_accuracy: 0.2049 - val_loss: 2.0478
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.2003 - loss: 1.9930 - val_accuracy: 0.1910 - val_loss: 1.9837
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.2271 - loss: 1.9284 - val_accuracy: 0.2812 - val_loss: 1.9165
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.2562 - loss: 1.8902 - val_accuracy: 0.2847 - val_loss: 1.8746
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 64ms/step - accuracy: 0.2701 - loss: 1.8448 - val_accuracy: 0.2708 - val_loss: 1.9073
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.3019 - loss: 1.7998 - val_accuracy: 0.2708 - val_loss: 1.9143
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.3068 - loss: 1.7847 - val_accuracy: 0.2951 - val_loss: 1.8649
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.3579 - loss: 1.7569 - val_accuracy: 0.2778 - v

In [2]:
!pip uninstall transformers -y
!pip install transformers

Found existing installation: transformers 4.50.2
Uninstalling transformers-4.50.2:
  Successfully uninstalled transformers-4.50.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 64.1 MB/s eta 0:00:00


In [14]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Set dataset path to Actor_01
actor_path = "/content/ravdess/Actor_01"

# List all audio files in the directory
audio_files = [f for f in os.listdir(actor_path) if f.endswith(".wav")]

# 🔥 Ensure we have audio files
if len(audio_files) == 0:
    raise ValueError(f"No .wav files found in '{actor_path}'. Please check the folder.")

# Define dummy text samples (one per audio file)
text_samples = [f"Sample text for {file}" for file in audio_files]

# Define labels dynamically (cycling through 0, 1, and 2)
labels = [i % 3 for i in range(len(audio_files))]

# ✅ Ensure all lists are the same length
if not (len(text_samples) == len(audio_files) == len(labels)):
    raise ValueError("Mismatch in text, audio, and label lengths.")

# Create dataset
data = {
    "text": text_samples,
    "audio_path": [os.path.join(actor_path, f) for f in audio_files],
    "label": labels
}

df = pd.DataFrame(data)

# ✅ Print dataset info for debugging
print(f"Total Audio Files in Actor_01: {len(audio_files)}")
print(f"Dataset Size: {len(df)}")
print(df.head())

# 🔥 Ensure dataset is not empty before splitting
if df.empty:
    raise ValueError("Dataset is empty after filtering. Check your files.")

# Split dataset
train_texts, test_texts, train_audio, test_audio, train_labels, test_labels = train_test_split(
    df["text"], df["audio_path"], df["label"], test_size=0.2, random_state=42
)

print("Dataset split successfully!")


Total Audio Files in Actor_01: 60
Dataset Size: 60
                                       text  \
0  Sample text for 03-01-06-01-01-02-01.wav   
1  Sample text for 03-01-08-01-01-01-01.wav   
2  Sample text for 03-01-08-02-01-02-01.wav   
3  Sample text for 03-01-08-01-02-02-01.wav   
4  Sample text for 03-01-05-01-02-01-01.wav   

                                          audio_path  label  
0  /content/ravdess/Actor_01/03-01-06-01-01-02-01...      0  
1  /content/ravdess/Actor_01/03-01-08-01-01-01-01...      1  
2  /content/ravdess/Actor_01/03-01-08-02-01-02-01...      2  
3  /content/ravdess/Actor_01/03-01-08-01-02-02-01...      0  
4  /content/ravdess/Actor_01/03-01-05-01-02-01-01...      1  
Dataset split successfully!


In [1]:
pip uninstall wandb -y


Found existing installation: wandb 0.19.8
Uninstalling wandb-0.19.8:
  Successfully uninstalled wandb-0.19.8


In [4]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [16]:
# Install necessary libraries
!pip install torch torchaudio transformers datasets librosa numpy pandas scikit-learn openai-whisper tensorflow keras

import os
import librosa
import numpy as np
import pandas as pd
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import TrainingArguments, Trainer

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Whisper ASR model
whisper_asr = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Function to transcribe audio using Whisper
def transcribe_audio(audio_path):
    try:
        transcription = whisper_asr(audio_path)
        return transcription["text"]
    except Exception as e:
        return ""

# Load RAVDESS dataset from folder
dataset_path = "/content/ravdess/Actor_01"
data = {"text": [], "audio_path": [], "label": []}

# Loop through the audio files
for file in os.listdir(dataset_path):
    if file.endswith(".wav"):
        audio_path = os.path.join(dataset_path, file)
        text = transcribe_audio(audio_path)
        label = int(file.split("-")[2])  # Extract emotion label
        data["text"].append(text)
        data["audio_path"].append(audio_path)
        data["label"].append(label)

# Convert to DataFrame
df = pd.DataFrame(data)

# Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)

# Load Transformer model for sentiment analysis
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
transformer_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=8).to(device)

# Custom Dataset Class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts.tolist()
        self.labels = labels.tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
        return {key: val.squeeze() for key, val in encoding.items()}, torch.tensor(label, dtype=torch.long)

# Prepare datasets
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
test_dataset = SentimentDataset(test_texts, test_labels, tokenizer)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Training setup
training_args = TrainingArguments(output_dir="./results", num_train_epochs=5, evaluation_strategy="epoch")
trainer = Trainer(model=transformer_model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)

# Train model
trainer.train()

# Evaluate on test set
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        logits = transformer_model(**inputs).logits
    return torch.argmax(logits).item()

# Test a sample
sample_text = "I am very happy today!"
predicted_label = predict_sentiment(sample_text)
print(f"Predicted Sentiment: {predicted_label}")


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The 

KeyboardInterrupt: 

In [17]:
# Install necessary libraries for realt time emotion prediction
!pip install transformers openai-whisper torch torchaudio librosa datasets scikit-learn pandas numpy matplotlib seaborn

import os
import torch
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from transformers import pipeline, BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

# Load Whisper ASR model
whisper_asr = pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Function to transcribe audio
def transcribe_audio(audio_path):
    transcription = whisper_asr(audio_path)
    return transcription["text"]

# Load RAVDESS dataset
data_dir = "/content/ravdess/Actor_01"  # Update path if needed
audio_files = []
labels = []

for file in os.listdir(data_dir):
    if file.endswith(".wav"):
        file_path = os.path.join(data_dir, file)
        emotion_label = int(file.split("-")[2])  # Extract emotion label from filename
        audio_files.append(file_path)
        labels.append(emotion_label)

# Transcribe audio to text
transcriptions = [transcribe_audio(audio) for audio in audio_files]

# Create DataFrame
df = pd.DataFrame({"text": transcriptions, "audio_path": audio_files, "label": labels})

# Split dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize transcribed text
train_encodings = tokenizer(list(train_texts), padding=True, truncation=True, return_tensors="pt")
test_encodings = tokenizer(list(test_texts), padding=True, truncation=True, return_tensors="pt")

# Convert labels to tensor
train_labels = torch.tensor(train_labels.values)
test_labels = torch.tensor(test_labels.values)

# Custom Dataset class
class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx],
        }

# Create Dataset objects
train_dataset = EmotionDataset(train_encodings, train_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)

# Load BERT model for sentiment classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8)

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train model
trainer.train()

# Save model
model.save_pretrained("./emotion_model")
tokenizer.save_pretrained("./emotion_model")

print("Training completed and model saved!")

# Load trained model for real-time predictions
loaded_model = BertForSequenceClassification.from_pretrained("./emotion_model")
loaded_tokenizer = BertTokenizer.from_pretrained("./emotion_model")

# Function to predict emotion from audio
def predict_emotion(audio_path):
    text = transcribe_audio(audio_path)
    inputs = loaded_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = loaded_model(**inputs)
    prediction = torch.argmax(outputs.logits).item()

    emotion_labels = {
        1: "Neutral", 2: "Calm", 3: "Happy", 4: "Sad",
        5: "Angry", 6: "Fearful", 7: "Disgust", 8: "Surprised"
    }

    return emotion_labels.get(prediction, "Unknown Emotion")

# Test real-time prediction
test_audio_path = "/content/ravdess/Actor_01/03-01-02-02-02-02-01.wav"  # Change file path accordingly
predicted_emotion = predict_emotion(test_audio_path)
print(f"Predicted Emotion: {predicted_emotion}")

# ---------------------
# Training Performance Visualization
# ---------------------

# Read training logs
logs = pd.read_json("./results/trainer_state.json")

# Extract accuracy and loss
train_loss = [entry["loss"] for entry in logs["log_history"] if "loss" in entry]
eval_loss = [entry["eval_loss"] for entry in logs["log_history"] if "eval_loss" in entry]
epochs = list(range(1, len(train_loss) + 1))

# Plot training loss vs evaluation loss
plt.figure(figsize=(10, 5))
plt.plot(epochs, train_loss, label="Training Loss", marker="o")
plt.plot(epochs, eval_loss, label="Evaluation Loss", marker="o")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training vs Evaluation Loss")
plt.legend()
plt.grid(True)
plt.show()

print("Training Performance Visualization Completed!")


Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


KeyboardInterrupt: 

In [22]:
import os
import torch
import librosa
import whisper
import warnings
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

# 🚀 Suppress warnings
warnings.filterwarnings("ignore")

# 🚀 Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# 🚀 Enable GPU acceleration
torch.backends.cudnn.benchmark = True

# 🚀 Use `Whisper Tiny.en` (Fastest)
whisper_model = whisper.load_model("tiny.en", device=device)

# Dataset path
ravdess_path = "/content/ravdess/Actor_01/"

# Extract features
data = {"text": [], "audio_features": [], "label": []}

for file in os.listdir(ravdess_path):
    if file.endswith(".wav"):
        file_path = os.path.join(ravdess_path, file)

        # 🚀 Load audio with reduced sample rate for speed
        audio, sr = librosa.load(file_path, sr=8000)

        # 🚀 Reduce MFCC features (5 instead of 13)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=5).mean(axis=1).astype(np.float32)

        # 🚀 Whisper-Tiny.en model (Ultra-fast)
        result = whisper_model.transcribe(file_path, fp16=(device == "cuda"))
        text = result["text"]

        # Extract emotion label from filename (RAVDESS format)
        emotion_label = int(file.split("-")[2])

        # Append data
        data["text"].append(text)
        data["audio_features"].append(mfcc)
        data["label"].append(emotion_label)

# Convert to DataFrame
df = pd.DataFrame(data)

# Encode labels
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

# 🚀 Train-test split
train_texts, test_texts, train_audio, test_audio, train_labels, test_labels = train_test_split(
    df["text"], df["audio_features"], df["label"], test_size=0.2, random_state=42
)

# 🚀 Fast Tokenizer (DistilBERT Fast)
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

# Convert to PyTorch Dataset
class MultimodalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, audio_features, labels):
        self.encodings = encodings
        self.audio_features = torch.tensor(np.stack(audio_features), dtype=torch.float32)
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["audio_features"] = self.audio_features[idx]
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# 🚀 DataLoader Optimization
train_loader = DataLoader(MultimodalDataset(train_encodings, train_audio, train_labels),
                          batch_size=32, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(MultimodalDataset(test_encodings, test_audio, test_labels),
                         batch_size=32, num_workers=8, pin_memory=True)

# 🚀 Load DistilBERT (default)
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                            num_labels=len(label_encoder.classes_))

# 🚀 Optimize Model for Speed
if torch.cuda.is_available():
    model = torch.compile(model)

# 🚀 Training setup (No evaluation, No logs)
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    evaluation_strategy="no",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    dataloader_num_workers=8,
    logging_strategy="no",
    save_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=MultimodalDataset(train_encodings, train_audio, train_labels)
)

# 🚀 Ultra-Fast Training
trainer.train()

# 🚀 Save Model
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")

print("🔥 Training Done! Extreme Speed 🚀🚀🚀")


100%|█████████████████████████████████████| 72.1M/72.1M [00:05<00:00, 15.1MiB/s]


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


🔥 Training Done! Extreme Speed 🚀🚀🚀


In [23]:
import torch
import librosa
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load trained model and tokenizer
model_path = "./sentiment_model"
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.eval()  # Set model to evaluation mode

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load Label Encoder (Ensure label_encoder is available from training)
import pickle
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Function to preprocess text
def process_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    return inputs

# Function to extract MFCC audio features
def process_audio(audio_path):
    audio, _ = librosa.load(audio_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=13).mean(axis=1).astype(np.float32)
    audio_tensor = torch.tensor(mfcc).unsqueeze(0).to(device)  # Add batch dimension
    return audio_tensor

# Test sample (Replace with actual test data)
test_text = "I am so excited for the event today!"  # Sample text input
test_audio_path = "/content/ravdess/Actor_01/03-01-06-01-02-01-01.wav"  # Sample audio file

# Preprocess text & audio
text_inputs = process_text(test_text)
audio_inputs = process_audio(test_audio_path)

# Predict emotion
with torch.no_grad():
    output = model(**text_inputs)
    pred_label = torch.argmax(output.logits, dim=1).item()

# Convert label to emotion name
predicted_emotion = label_encoder.inverse_transform([pred_label])[0]
print(f"Predicted Emotion: {predicted_emotion} 🎭")


FileNotFoundError: [Errno 2] No such file or directory: 'label_encoder.pkl'

In [25]:
import os

if not os.path.exists("label_encoder.pkl"):
    print("⚠️ label_encoder.pkl not found! Run the training code first.")
else:
    print("✅ label_encoder.pkl found. Proceeding with testing...")


⚠️ label_encoder.pkl not found! Run the training code first.


In [26]:
import pickle
from sklearn.preprocessing import LabelEncoder

# Recreate & Save Label Encoder
label_encoder = LabelEncoder()

# Define labels manually if needed (adjust based on your dataset)
possible_labels = list(range(1, 9))  # Assuming RAVDESS dataset has 8 emotions
label_encoder.fit(possible_labels)

# Save label encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("✅ Label Encoder saved as 'label_encoder.pkl'")


✅ Label Encoder saved as 'label_encoder.pkl'


In [27]:
#Prediction on the above trained model
import torch
import librosa
import whisper
import pickle
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Load Whisper model
whisper_model = whisper.load_model("base")

# Load Trained Model
model_path = "./sentiment_model"
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)

# Load Label Encoder
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

print("✅ Model & Label Encoder Loaded!")

# Function to predict sentiment
def predict_sentiment(audio_path):
    # Convert speech to text
    result = whisper_model.transcribe(audio_path)
    text = result["text"]

    # Extract audio features (MFCC)
    audio, sr = librosa.load(audio_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13).mean(axis=1)

    # Tokenize text
    encoding = tokenizer(text, truncation=True, padding=True, max_length=128, return_tensors="pt")

    # Make prediction
    model.eval()
    with torch.no_grad():
        output = model(**encoding)
        pred_label = torch.argmax(output.logits, dim=1).item()

    # Convert label index back to emotion
    emotion = label_encoder.inverse_transform([pred_label])[0]

    return text, emotion

# Test on a new audio file
test_audio_path = "/content/ravdess/Actor_01/03-01-01-01-01-01-01.wav"  # Replace with an actual audio file path
text_output, predicted_emotion = predict_sentiment(test_audio_path)

print(f"🎙️ Transcribed Text: {text_output}")
print(f"🔮 Predicted Emotion: {predicted_emotion}")


✅ Model & Label Encoder Loaded!
🎙️ Transcribed Text:  Kids are talking by the door.
🔮 Predicted Emotion: 8
